# TSA Chapter 5: VAR Rolling Forecast

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_var_rolling_forecast/TSA_ch5_var_rolling_forecast.ipynb)

Rolling one-step-ahead VAR forecasts for a bivariate system.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas-datareader -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, grangercausalitytests, acf
import pandas_datareader.data as web
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Simulate bivariate VAR(1) process
np.random.seed(42)
A = np.array([[0.5, 0.3], [-0.2, 0.4]])
n = 300; Y = np.zeros((n, 2))
for t in range(1, n):
    Y[t] = A @ Y[t-1] + np.random.normal(0, 1, 2)

data = pd.DataFrame(Y, columns=['$Y_1$', '$Y_2$'])

# Rolling one-step-ahead forecast
n_test = 100; n_train = n - n_test
forecasts = {col: [] for col in data.columns}
for i in range(n_test):
    train = data.iloc[:n_train + i]
    try:
        res = VAR(train).fit(1)
        fcast = res.forecast(train.values[-1:], steps=1)
        for j, col in enumerate(data.columns):
            forecasts[col].append(fcast[0, j])
    except:
        for col in data.columns:
            forecasts[col].append(0)

# Compute RMSE
actual_1 = data['$Y_1$'].values[n_train:n_train + n_test]
actual_2 = data['$Y_2$'].values[n_train:n_train + n_test]
rmse_1 = np.sqrt(np.mean((actual_1 - np.array(forecasts['$Y_1$']))**2))
rmse_2 = np.sqrt(np.mean((actual_2 - np.array(forecasts['$Y_2$']))**2))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
test_idx = range(n_train, n_train + n_test)

axes[0].plot(test_idx, actual_1, color=BLUE, lw=1.2, label='Actual')
axes[0].plot(test_idx, forecasts['$Y_1$'], color=ORANGE, lw=1.2, ls='--', label='1-step Forecast')
axes[0].set_title(f'Rolling Forecast: $Y_1$ (RMSE={rmse_1:.3f})', fontweight='bold')
axes[0].set_xlabel('Time'); axes[0].set_ylabel('Value')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

axes[1].plot(test_idx, actual_2, color=RED, lw=1.2, label='Actual')
axes[1].plot(test_idx, forecasts['$Y_2$'], color=ORANGE, lw=1.2, ls='--', label='1-step Forecast')
axes[1].set_title(f'Rolling Forecast: $Y_2$ (RMSE={rmse_2:.3f})', fontweight='bold')
axes[1].set_xlabel('Time'); axes[1].set_ylabel('Value')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

plt.tight_layout(); save_chart(fig, 'ch5_var_rolling_forecast'); plt.show()